Project Ideas


suggested project: analyze wind speed around the country with a view to a wind farm


## Project Plan

__Research wind farms in Ireland__

- where are they usually located?
- what wind conditions do they need? any other site considerations? Rural locations.
- how much electricity do they generate? summer vs winter?
- cost?
- lifespan?
- advantages
- disadvantages
- anything else?

__Project questions__

What's the relationship between wind speed and power generated? Does the wind direction affect power generation? 

Is there a trend in wind speed? Is Ireland getting winder? Variations across the year? Time of day?

Is the technology in wind turbines improving? Is more electricity being generated for the same wind speed?

Does rain/temperature/anything affect the output? 

What happens during a storm? Does amount of wind generated electricity decrease/increase? 

Predict power output for wind farms in Ireland for the next week. Tricky

As I have weather information could solar power to fill the gaps when wind speeds are low? Probably too big a task for this project. 


__Find data__

Weather data from met Éireann historical data.
    can select by site, perhaps initially analyse data for a number of weather stations near a wind farm and also weather stations not near a wind farm. From the data can I see why that site was selected?



Is there much variation in wind across the country? Eirgrid data for entire country. 

## Introduction

Background information

https://windenergyireland.com/about-wind/the-basics/facts-stats

## The Data

About the data set. 

## Organising and Cleaning the Data


Would be convenient to have all the data in one large data set. Need to research working with large data sets. More difficult to load than smaller data sets.

Clean data

In [2]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import re

## Wind electricity data

https://www.smartgriddashboard.com/#all

[Eirgrid real-time system information](https://www.eirgrid.ie/grid/real-time-system-information) . On the Eirgrid website it is only possible to view information for one day at a time and up to one month ago. Despite extensive searching I couldn't find an official source of Eirgrid histprical data. I did find a [GitHub repository by Daniel Parke](https://github.com/Daniel-Parke/EirGrid_Data_Download/tree/main), who has written a very helpful python file to download all the historical data. His GitHub repository contains raw csv files for actual amount of electricity generated, actual demand, actual amount of electricity produced by wind for every year from 2014 for all Ireland, Northern Ireland and Republic of Ireland. I will need to run his program to get the most up to date data for 2024.

As my weather data will be only for the Republic of Ireland, I am only interested in the csv files for the actual amount of electricity produced by wind for the Republic of Ireland. Each csv file containing one years worth of information was downloaded from the GitHub repository. After reading the data into pandas the next task will be to merge the files vertically using pd.concat(). Before I started on the onerous task of loading and concatenating each file separately, I came across a blog post on how to [read multiple csv file into pandas](https://saturncloud.io/blog/how-to-read-multiple-csv-files-into-python-pandas-dataframe). 

The solution to reading multiple files into pandas uses the glob module. Glob is a built-in module used to retrieve files/pathnames matching a specified pattern. It uses * wild cards to make path retrieval more simple and convenient. https://www.geeksforgeeks.org/how-to-use-glob-function-to-find-files-recursively-in-python/. [Real python](https://realpython.com/get-all-files-in-directory-python/#conditional-listing-using-glob) states that glob.glob() returns a list of filenames that match a pattern, which in this case are csv files. 

```python
# Search for all csv files in the current working directory
import glob
glob.glob('*.csv')
```

Eirgrid have data on actual wind generation and the forecast wind generation. Could getting forecast information be of interest? Might help with machine learning. Github repository only contains actual data not forecast data. 

set up a scheduled task to download the data at midnight?


In [ ]:
## This was the first step to create a single dataframe from multiple csv files. 
## 
## Load the wind electricity files
#
## Find all csv files in the data/electricity directory
#csv_files = glob.glob('data/electricity/*.csv')
#
## Create an empty dataframe to store the combined data
#electricity_df = pd.DataFrame()
#
## Loop through each CSV file and append its contents to the combined dataframe
#for csv_file in csv_files:
#    df = pd.read_csv(csv_file, 
#                     header = None, 
#                     names = ['date', 'wind_actual', 'location', 'wind_value'], 
#                     index_col= 'date',
#                     parse_dates= ['date'],
#                     usecols= ['date', 'wind_value'])
#    electricity_df = pd.concat([electricity_df, df])
#
#electricity_df.head()

```python
electricity_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 361152 entries, 2021-01-01 00:00:00 to 2020-01-01 21:45:00
Data columns (total 1 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   wind_value  361004 non-null  float64
dtypes: float64(1)
memory usage: 5.5 MB
```

In [ ]:
electricity_df.info()

In [ ]:
electricity_df.isna().sum()

have a lot of duplicated rows. csv files have data for the 1st jan for the following year. 

In [ ]:
#electricity_df.index.duplicated().sum()

In [ ]:
# https://stackoverflow.com/questions/13035764/remove-pandas-rows-with-duplicate-indices
#
#electricity_df = electricity_df[~electricity_df.index.duplicated(keep= 'first')]
#electricity_df.head()

In [ ]:
#electricity_df = electricity_df.sort_index()

In [ ]:
#electricity_df.shape

In [ ]:
#electricity_df.to_csv('data/electricity/electricity_data.csv')

In [ ]:
hourly_electricity_df = electricity_df.resample('h').mean()
hourly_electricity_df.head()

In [ ]:
hourly_electricity_df.to_csv('data/electricity/hourly_electricity.csv')

In [ ]:
hourly_electricity_df = pd.read_csv('data/electricity/merged_data/hourly_electricity.csv')
hourly_electricity_df.head()

## Weather Data

The weather data was downloaded from historic met eireann sites. Data from a range of weather stations was downloaded. The selected weather stations were mostly based on their proximity to a wind farm. A number were selected for the size of the data set. To see if Ireland is getting windier. Oldest weather stations with data are Dublin airport and Valentia who have data from 1 Jan 1944. 

Met Eireann weather data is recorded hourly. Electricity data recorded every 15min - resample to 1 hour. Saved resampled data to hourly_electricity.csv Should I read in this single csv file? 


Weather Data

What I'm aiming to do

Look at wind speeds for the entire country. Is there much variability? Electricity generation values for the entire country. 

    Read all the csv files in weather directory into pandas.

        Problems: some csv files have a different numbers of rows to skip. Function to remove the unnecessary rows from csv file. 

        The dataframe would ideally be the location. This is proving very difficult. Have written a function to extract the location from file name. 

    Refine the dataframe to the years 2014 onwards. Write a function. 

    Merge the dataframes
        Can all the merging be done in one step? Write a function. 

Analyse the data. 


### Is there much variation in wind speed across the country? 

Electricity generated is given for ROI not broken down by wind farm.

Electricity data from 2014, so to compare wind speed and amount of electricity generated by wind only need weather data from 2014. 


Write a function to read in the weather data. Basic elements are the same skip rows, na_values, use columns, parse_dates

What columns are needed? 
date, rain, temp, msl, wdsp, wddir, 

sun, clht, clamt not recorded for all weather stations. so not of interest. 

Remove explanatory rows in csv file. 

In [3]:
def skip_rows(csv_file):
    # Read the file, skipping metadata rows
    with open(csv_file, 'r') as file:
        lines = file.readlines()
    
    # Identify the start of the data (row where actual CSV content begins)
    for i, line in enumerate(lines):
        if line.lower().startswith('date,'):
            data_start_idx = i
    
    return data_start_idx

In [4]:
skip_rows('data/weather/hly275MaceHead.csv')

17

In [5]:
skip_rows('data/weather/hly518ShannonAirport.csv')

23

In [6]:
def extract_location(file_name):
    '''A function to extract the location from the file name'''

    pattern = r'hly\d{3,4}([A-Z][a-z]+[A-Z]?[a-z]+).csv'

    match = re.findall(pattern, file_name)

    if match:
        return match[0].lower()
    else:
        raise ValueError('File name does not match the expected pattern')


In [7]:
# Extract the location from the csv files in weather
csv_files = glob.glob('data/weather/*.csv')

location = []

for file in csv_files:
    name = extract_location(file)
    location.append(name)
print(location)

['johnstown', 'mullingar', 'dublinairport', 'athenry', 'gurteen', 'shannonairport', 'macehead', 'finner', 'corkairport', 'valentia', 'belmullet']


In [8]:
csv_files = glob.glob('data/weather/*.csv')

l = []

for filename in csv_files:
    for skip_rows in [17, 23]:
        try:
            df = pd.read_csv(filename, skiprows= skip_rows, na_values= ' ', usecols= ['date', 'rain', 'temp', 'msl', 'wdsp', 'wddir'], index_col= 'date', parse_dates= ['date'], date_format = "%d-%b-%Y %H:%M")
            l.append(df)
        except Exception as e:
            continue

print(l)

[                     rain  temp     msl  wdsp  wddir
date                                                
2003-08-12 01:00:00   NaN   NaN     NaN   NaN    NaN
2003-08-12 02:00:00   NaN   NaN     NaN   NaN    NaN
2003-08-12 03:00:00   NaN   NaN     NaN   NaN    NaN
2003-08-12 04:00:00   NaN   NaN     NaN   NaN    NaN
2003-08-12 05:00:00   NaN   NaN     NaN   NaN    NaN
...                   ...   ...     ...   ...    ...
2024-10-31 20:00:00   0.0  11.6  1025.4   5.0  260.0
2024-10-31 21:00:00   0.0  11.5  1025.2   4.0  270.0
2024-10-31 22:00:00   0.0  11.5  1025.1   3.0  270.0
2024-10-31 23:00:00   0.0  11.5  1024.9   3.0  260.0
2024-11-01 00:00:00   0.0  11.6  1024.7   4.0  270.0

[186048 rows x 5 columns],                      rain  temp     msl  wdsp  wddir
date                                                
1973-11-07 01:00:00   0.0   5.0  1030.3   7.0  220.0
1973-11-07 02:00:00   0.0   5.3  1029.8   7.0  220.0
1973-11-07 03:00:00   0.0   6.0  1029.3   9.0  230.0
1973-11-07 04:00:

In [9]:
weather_df = pd.concat(l, axis= 1)

In [10]:
weather_df.head()

,rain,temp,msl,wdsp,wddir,rain,temp,msl,wdsp,wddir,...,rain,temp,msl,wdsp,wddir,rain,temp,msl,wdsp,wddir
date,,,,,,,,,,,,,,,,,,,,,
1944-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,9.3,1034.3,12.0,300.0,NaN,NaN,NaN,NaN,NaN
1944-01-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,8.9,1033.9,9.0,290.0,NaN,NaN,NaN,NaN,NaN
1944-01-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,9.4,1033.4,11.0,280.0,NaN,NaN,NaN,NaN,NaN
1944-01-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,9.3,1032.8,11.0,280.0,NaN,NaN,NaN,NaN,NaN
1944-01-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.5,8.6,1032.4,11.0,300.0,NaN,NaN,NaN,NaN,NaN


In [11]:
weather_df = weather_df['2014' :'2024']

In [13]:
weather_df.head()

,rain,temp,msl,wdsp,wddir,rain,temp,msl,wdsp,wddir,...,rain,temp,msl,wdsp,wddir,rain,temp,msl,wdsp,wddir
date,,,,,,,,,,,,,,,,,,,,,
2014-01-01 00:00:00,0.0,7.6,992.4,13.0,220.0,0.0,3.4,989.8,8.0,180.0,...,0.0,7.5,988.7,9.0,180.0,0.0,7.1,984.0,17.0,210.0
2014-01-01 01:00:00,0.0,6.6,992.4,11.0,230.0,0.0,3.0,989.7,7.0,180.0,...,0.0,7.2,987.5,7.0,180.0,0.0,6.4,983.7,17.0,200.0
2014-01-01 02:00:00,0.0,6.7,991.7,9.0,210.0,0.0,3.0,989.1,7.0,180.0,...,0.3,8.4,986.2,9.0,180.0,0.1,7.2,983.2,15.0,200.0
2014-01-01 03:00:00,0.0,7.5,991.1,10.0,210.0,0.1,3.0,988.5,7.0,170.0,...,1.6,7.1,984.4,9.0,180.0,0.1,7.0,982.5,16.0,190.0
2014-01-01 04:00:00,0.0,8.0,989.6,11.0,200.0,0.0,2.8,987.4,7.0,160.0,...,0.8,7.7,981.9,8.0,160.0,1.0,6.6,981.6,16.0,190.0


In [17]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 94969 entries, 2014-01-01 00:00:00 to 2024-11-01 00:00:00
Freq: h
Data columns (total 55 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   rain    94940 non-null  float64
 1   temp    94941 non-null  float64
 2   msl     94941 non-null  float64
 3   wdsp    94935 non-null  float64
 4   wddir   94939 non-null  float64
 5   rain    94959 non-null  float64
 6   temp    94961 non-null  float64
 7   msl     94961 non-null  float64
 8   wdsp    94956 non-null  float64
 9   wddir   94959 non-null  float64
 10  rain    94969 non-null  float64
 11  temp    94969 non-null  float64
 12  msl     94969 non-null  float64
 13  wdsp    94969 non-null  int64  
 14  wddir   94969 non-null  float64
 15  rain    94956 non-null  float64
 16  temp    94957 non-null  float64
 17  msl     94869 non-null  float64
 18  wdsp    94943 non-null  float64
 19  wddir   94943 non-null  float64
 20  rain    94932 non-null  float

In [20]:
#https://stackoverflow.com/questions/68233237/computing-row-average-of-columns-with-same-name-in-pandas
weather_df_mean = weather_df.T.groupby(weather_df.columns).mean().T

In [21]:
weather_df_mean

,msl,rain,temp,wddir,wdsp
date,,,,,
2014-01-01 00:00:00,988.718182,0.036364,5.772727,190.909091,12.363636
2014-01-01 01:00:00,988.390909,0.027273,5.281818,186.363636,10.909091
2014-01-01 02:00:00,987.736364,0.036364,5.590909,176.363636,10.545455
2014-01-01 03:00:00,986.854545,0.209091,5.472727,171.818182,10.181818
2014-01-01 04:00:00,985.463636,0.200000,5.590909,160.000000,10.545455
...,...,...,...,...,...
2024-10-31 20:00:00,1024.636364,0.000000,11.127273,207.272727,5.545455
2024-10-31 21:00:00,1024.345455,0.000000,11.118182,211.818182,5.727273
2024-10-31 22:00:00,1024.245455,0.000000,11.090909,210.909091,5.272727


In [24]:
weather_df_mean.to_csv('data/weather/merged_data/weather_mean.csv')

In [ ]:
# Extract the location from the csv files in weather
csv_files = glob.glob('data/weather/*.csv')

location_dict = {}

for file in csv_files:
    name = extract_location(file)
    location_dict[name + '_df'] = file
print(location_dict)


In [ ]:
def load_weather(file_name):
    '''A function to read in a weather csv file.'''
       
    file_path = 'data/weather/'

    for skip_rows in [17, 23]:
        weather_df = pd.read_csv(file_path + file_name,
            skiprows = skip_rows,
            usecols= ['date', 'rain', 'temp', 'msl', 'wdsp', 'wddir'],
            na_values = ' ',
            index_col= 'date', 
            parse_dates= ['date'], 
            date_format = "%d-%b-%Y %H:%M")
            
        weather_df = weather_df['2014': '2024']

    return weather_df

#date, rain,temp,msl,wdsp,wddir

In [ ]:
def read_and_filter_csv(file_path):

    file_path = 'data/weather/'

    for skip_rows in [17, 23]:  # Try skipping 17 or 23 rows
        try:
            df = pd.read_csv(file_path + file_path, skiprows=skip_rows)
            return df
        except Exception as e:
            continue  # Try the next skiprows value if an error occurs
    
    # Return an empty DataFrame if neither option works
    return pd.DataFrame()

In [ ]:
gurteen = load_weather('hly1475Gurteen.csv')
gurteen.head()

In [ ]:
macehead = load_weather('hly275MaceHead.csv')
macehead.head()

In [ ]:
mullingar = load_weather('hly875Mullingar.csv', 17)

In [ ]:
johnstown = load_weather('hly1775Johnstown.csv', 17)

In [ ]:
athenry = load_weather('hly1875Athenry.csv', 17)

In [ ]:
finner = load_weather('hly2075Finner.csv', 17)

refine dataframes to 2014 data onwards

In [ ]:
def select_years(df):
    df = df['2014': '2024']
    return df

Practice merging dataframes

In [ ]:
gurteen_macehead = gurteen.merge(macehead, on= 'date', suffixes= ['_gur', '_mace'])
gurteen_macehead.head()

In [ ]:
location = extract_location('hly275MaceHead.csv')
print(location)

In [ ]:
location = extract_location('hly1475Gurteen.csv')
print(location)

In [ ]:
gurteen = load_weather('hly1475Gurteen.csv', 17)

In [ ]:
lo

regex the filename
 # Want df to have the station name in the name of the df
 #pattern = r'hly\d{3,4}([A-Z][a-z]+).csv'
 #my_string = file_name
 #match = re.search(pattern, my_string)
 #name = match.group(1).lower()
 #name_df = f"{name}_df"



In [ ]:
type(name_df)

In [ ]:
# Want to the file name to name the dataframe. 

In [ ]:
name_df.head()

In [ ]:
name_df = name('hly275MaceHead.csv')

df_name('hly875Mullingar.csv', 17)

In [ ]:
mullingar_df.head()

In [ ]:
name('hly875Mullingar.csv')

In [ ]:
# Create a data frame with name extracted from the csv file
name_df = name('hly875Mullingar.csv')

name_df = load_weather('hly875Mullingar.csv', 17)


In [ ]:
name_df = name('hly2375Belmullet.csv')
name_df = load_weather('hly2375Belmullet.csv', 23)

In [ ]:
belmullet_df = pd.read_csv('data/weather/hly2375Belmullet.csv', 
                           skiprows = 23,
                           usecols= ['date', 'rain', 'temp', 'msl', 'wdsp', 'wddir'],
                           na_values = ' ',
                           index_col= 'date', 
                           parse_dates= ['date'], 
                           date_format = "%d-%b-%Y %H:%M")

belmullet_df.head()

In [ ]:
# Select rows from 2014

belmullet_df = belmullet_df.loc['2014' : '2024']

belmullet_df.head()

Merge weather dataframes



### Is Ireland getting windier? Use Dublin Airport data. Recorded from 1944. Also Valentia recorded from then too. Do Dublin first.

In [ ]:
dublin_df = pd.read_csv('data/weather/hly532DublinAirport.csv', 
                        skiprows = 23, 
                        na_values = ' ',
                        index_col= 'date', 
                        parse_dates= ['date'], 
                        date_format = "%d-%b-%Y %H:%M")

dublin_df.head()

In [ ]:
dublin_df.info()

In [ ]:
dublin_df = dublin_df.dropna()

In [ ]:
dublin_df.info()

In [ ]:
dublin_monthly = dublin_df.resample('ME')

In [ ]:
fig, ax = plt.subplots(figsize = (15, 5))

dublin_yearly = dublin_df.resample('YE')


dublin_df['wdsp'].resample('YE').mean().scatterplot()

plt.show()

In [ ]:
print(dublin_yearly['wdsp'].mean())

In [ ]:
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from matplotlib.pylab import rcParams

rcParams['figure.figsize'] = 11, 9

decomposition = sm.tsa.seasonal_decompose(dublin_monthly['wdsp'].mean(), model= 'additive', period = 12)
fig = decomposition.plot()


In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose, STL , MSTL
#from statsforecast import StatsForecast
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
valentia_df = pd.read_csv('data/weather/hly2275Valentia.csv', 
                        skiprows = 23, 
                        na_values = ' ',
                        index_col= 'date', 
                        parse_dates= ['date'], 
                        date_format = "%d-%b-%Y %H:%M")

valentia_df.head()

In [ ]:
fig, ax = plt.subplots(figsize = (15, 5))

valentia_df['wdsp'].resample('YE').mean().plot()

plt.show()

In [ ]:
valentia_monthly = valentia_df.resample('ME')

In [ ]:
rcParams['figure.figsize'] = 11, 9

decomposition = sm.tsa.seasonal_decompose(valentia_monthly['wdsp'].mean(), model= 'additive', period = 12)
fig = decomposition.plot()


## Exploratory Data Analysis

In [ ]:
# Very quick plot of electricity generation by year

mean_wind_elect_year = electricity_df.resample('YE').mean()

mean_wind_elect_year.plot()
plt.show()

## Discussion of the Analysis

nice plots

## Machine Learning

some predictive analysis.

## Conclusion



## References


http://www.iwea.ie/technicalfaqs


### Data Sets

[GitHub Daniel Parke]https://github.com/Daniel-Parke/EirGrid_Data_Download/tree/main


__Problems that arose__

[Git LFS (large file storage)](https://git-lfs.com/). Some of the weather data filew were larger than GitHub's recommended maximum file size of 50.00 MB. Installed and used Git lfs
